In [209]:
import pandas as pd
import numpy as np

In [210]:
club_id_to_query = 10

In [211]:
# Load data
df_club_users = pd.read_csv('updated-data-set/clubs_club_users.csv')
df_club_books = pd.read_csv('updated-data-set/clubs_club_books.csv')
df_books = pd.read_csv('updated-data-set/clubs_book.csv', encoding = 'latin1')
df_book_ratings = pd.read_csv('BX-Book-Ratings.csv', encoding = 'latin1', sep = ';')

In [212]:
# Get id of all rows with a book rating of 0
df_book_ratings.drop(df_book_ratings[df_book_ratings['Book-Rating'] == 0].index, inplace = True)

In [213]:
# Find the number of ratings made by each user
df_rating_count = pd.DataFrame(df_book_ratings.groupby('User-ID')['Book-Rating'].count())

# Make Rating count as a regular column
df_rating_count.reset_index(level=0, inplace=True)

# Remove from the ratings table, all users with less than 20 ratings
df_rating_count.drop(df_rating_count[df_rating_count['Book-Rating'] < 20].index, inplace = True)
df_rating_count.drop('Book-Rating', axis = 1, inplace=True)

df_book_ratings = pd.merge(df_book_ratings, df_rating_count, on = 'User-ID')
df_book_ratings

,User-ID,ISBN,Book-Rating
0,276847,3257200552,5
1,276847,3404145909,8
2,276847,3404148576,8
3,276847,3404921178,7
4,276847,3423071516,10
...,...,...,...
246793,276688,0836218655,10
246794,276688,0836236688,10
246795,276688,0892966548,10
246796,276688,1551669315,6


In [214]:
# Get the favourite books of the club specified
df_favourite_books = pd.DataFrame(df_club_books['book_id'][df_club_books['club_id'] == club_id_to_query])
df_favourite_books

,book_id
45,289
46,108
47,331
48,421
49,131


In [215]:
# Get ISBNs of the club's favourite books
df_favourite_books = pd.merge(df_favourite_books, df_books, left_on = 'book_id', right_on = 'id')

# Get ISBNs and authors of the favourite books
fav_authors = df_favourite_books['author'].tolist()
fav_books_ISBNs = df_favourite_books['ISBN'].tolist()
df_favourite_books.head(5)

,book_id,id,ISBN,title,author,publisher,publication_year
0,289,289,0140285563,The Diary of a Nobody (Essential.penguin S.),George Grossmith,Penguin Books Ltd,1999
1,108,108,0060192321,Expedition Whydah: The Story of the World's Fi...,Barry Clifford,Cliff Street Books,1999
2,331,331,0393703703,"E-Therapy: Case Studies, Guiding Principles, a...",Robert C. Hsiung,W.W. Norton & Company,2002
3,421,421,0821773801,Midnight Sun,Kat Martin,Zebra Books,2003
4,131,131,0679441700,The Run of His Life: The People V. O.J. Simpson,Jeffrey Toobin,Random House Inc,1996


In [216]:
# Get all books by the favourite authors
df_all_author_books = pd.DataFrame()

for author in fav_authors:
    df_author_books = pd.DataFrame(df_books['ISBN'][df_books['author'] == author])
    # Exclude the books that are from the club's favourite books
    df_author_books = df_author_books[~df_author_books.ISBN.isin(df_favourite_books.ISBN)]
    df_all_author_books = pd.concat([df_all_author_books, df_author_books])

df_all_author_books.head(10)

,ISBN


In [217]:
# Get the most rated books from the above list
df_author_book_ratings = pd.merge(df_all_author_books, df_book_ratings, on = 'ISBN')
df_author_books_rating_count = pd.DataFrame(df_author_book_ratings.groupby('ISBN')['Book-Rating'].count())

# Make Rating count as a regular column and sort
df_author_books_rating_count.reset_index(level=0, inplace=True)
df_author_books_rating_count.sort_values('Book-Rating', ascending = False)

recommended_books = df_author_books_rating_count['ISBN'].iloc[0:10]
recommended_books = pd.merge(recommended_books, df_books, on = 'ISBN')
